In [1]:
MIN_NUMBER_OF_ACTIVE_MONTHS = 6
ALERT_TIME = 1 # Cantidad de meses esperado de anticipación antes de cancelar

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [3]:
data = pd.read_csv("../Churn Brasil_plana mensualizada.csv", dtype={'monto_total': object})

/home/fvadell/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
fede = pd.read_csv("../Churn Brasil_plana mensualizada.csv", dtype={'monto_total': object})

### Variables explicativas

Iniciamos las variables explicativas como todas las columnas del dataset e iremos sacando las que no sirven

#### Caso Monto_total

Sacamos las ','

In [5]:
data['monto_total'] = data['monto_total'].str.replace(',', '')

In [6]:
data['monto_total']=data['monto_total'].astype(float)

In [7]:
print('**** monto_total')
print('Min:{}'.format(data.monto_total.min()))
print('Max:{}'.format(data.monto_total.max()))

**** monto_total
Min:-39425.0
Max:93000.0


In [8]:
variables = list(data.columns)

Le saco la variable __Status Cliente__ porque es el target.

In [9]:
variables.remove('Status Cliente')

Saco las variables pais, ano_alta, idclientecrm, User vendedor

In [10]:
for var in ['pais', 'ano_alta', 'idclientecrm', 'User vendedor'] : variables.remove(var)

#### Caso de la columna tipo_cliente y CLIENTE GRUPO

Estas dos columnas parecen tener la misma información. Lo revisos:

In [11]:
data['tipo_cliente'].unique(), data['CLIENTE GRUPO'].unique()

(array(['Small', 'Big', 'definir'], dtype=object),
 array(['Small', 'Big', 'Desarrollo'], dtype=object))

Porcentaje de coincidencias

In [12]:
np.where(data['tipo_cliente'] == data['CLIENTE GRUPO'], True, False).sum()*100/len(data)

96.757159423812

Por lo tanto, usaremos la columna __CLIENTE GRUPO__ porque tiene prácticamente la misma información que __tipo_cliente__

In [13]:
variables.remove('tipo_cliente')

In [14]:
label_references = pd.DataFrame(columns = ['class', 'label'])

In [15]:
def generate_and_append_labels(class_list, label_reference_df):
    assert(set(label_reference_df.columns)==set(['class', 'label']) )
    l = pd.DataFrame(columns = ['class', 'label'])
    labelencoder = preprocessing.LabelEncoder()
    labelencoder.fit(class_list)
    l['class'] = class_list
    l['label'] = l['class'].apply(lambda x: labelencoder.transform([x])[0])
    label_reference_df = pd.concat([label_reference_df, l])
    assert(len(label_reference_df)==len(label_reference_df['class'].unique())) # Classes repetidad
    return label_reference_df

def apply_label(data:pd.DataFrame, column_name, label_df):
    assert(set(label_df.columns)==set(['class', 'label']))
    if column_name+'_label' in data.columns:
        print("Column already labeled")
        return
    data = pd.merge(data, label_references, left_on = column_name, right_on = 'class').drop('class', axis = 1)
    data = data.rename(columns={'label': column_name +'_label'})
    return data.drop(column_name, axis = 1)

In [16]:
label_references = generate_and_append_labels(data['CLIENTE GRUPO'].unique(), label_references)

In [17]:
data = apply_label(data, column_name = 'CLIENTE GRUPO', label_df = label_references)

In [18]:
variables.append('CLIENTE GRUPO_label')

In [19]:
variables.remove('CLIENTE GRUPO')

#### Caso Lead Alquiler Formulario' 'Lead Alquiler Ver Dato' 'Lead Alquiler temporal Formulario' 'Lead Alquiler temporal Ver Dato

Estas columnas tienen un '  -   ' en vez de 0. Vamos a reemplazarlos por 0.

In [20]:
for column in ['Lead Venta Formulario', 'Lead Venta Ver Dato','Lead Alquiler Formulario', 'Lead Alquiler Ver Dato','Lead Alquiler temporal Formulario', 'Lead Alquiler temporal Ver Dato']:
    print(column, end = '\r')
    data[column] = data[column].astype(str)
    data[column] = data[column].replace('  -   ' ,'0')
    data[column] = data[column].str.replace(',', '')
    data[column] = data[column].astype(float)
    print(column + " - OK!")

Lead Venta Formulario - OK!
Lead Venta Ver Dato - OK!
Lead Alquiler Formulario - OK!
Lead Alquiler Ver Dato - OK!
Lead Alquiler temporal Formulario - OK!
Lead Alquiler temporal Ver Dato - OK!


Además vamos a unir las de Formulario con las Ver Dato

In [21]:
data['Lead Venta'] = data['Lead Venta Formulario'] + data['Lead Venta Ver Dato']
variables.append('Lead Venta')
data['Lead Alquiler'] = data['Lead Alquiler Formulario'] + data['Lead Alquiler Ver Dato']
variables.append('Lead Alquiler')
data['Lead Alquiler temporal'] = data['Lead Alquiler temporal Formulario'] + data['Lead Alquiler temporal Ver Dato']
variables.append('Lead Alquiler temporal')

In [22]:
for x in ['Lead Venta Formulario', 'Lead Venta Ver Dato', 'Lead Alquiler Formulario', 'Lead Alquiler Ver Dato', 'Lead Alquiler temporal Ver Dato', 'Lead Alquiler temporal Formulario']:
    print(x, end = "\r")
    variables.remove(x)
    print(x + " - Removed")

Lead Venta Formulario - Removed
Lead Venta Ver Dato - Removed
Lead Alquiler Formulario - Removed
Lead Alquiler Ver Dato - Removed
Lead Alquiler temporal Ver Dato - Removed
Lead Alquiler temporal Formulario - Removed


#### Caso columna Integrador

Toma valores 'SI' y 'NO' que cambio por '1' y '0' respectivamente

In [23]:
label_references = generate_and_append_labels(data['Integrador'].unique(), label_references)

In [24]:
data = apply_label(data, column_name = 'Integrador', label_df = label_references)

In [25]:
variables.remove('Integrador')
variables.append('Integrador_label')

#### Caso Status Cliente

In [26]:
data['Status Cliente'].unique()

array(['Vigente', 'Cancelado', '-'], dtype=object)

In [27]:
data = data[data['Status Cliente']!='-']

In [28]:
data['Status Cliente'].value_counts()

Vigente      178020
Cancelado    158268
Name: Status Cliente, dtype: int64

In [29]:
len(data.idclientecrm.unique())

19748

#### Caso columnas contratado

Hay algunos 'NaN' en las columnas de contratados. Las cambio por 0.

In [30]:
contratados = ['contratado Super Destacado','contratado Destacado',
 'contratado Simple',
 'contratado Desarrollo Super Destacado',
 'contratado Desarrollo Destacado',
 'contratado Desarrollo Simple',
 'contratado freemium']

In [31]:
for c in contratados :
    data[c] = data[c].fillna(0)
    print(c+' con información relevante: ', len(data[(data[c]!='0') & (data[c]!=0)]))

contratado Super Destacado con información relevante:  140439
contratado Destacado con información relevante:  192742
contratado Simple con información relevante:  221376
contratado Desarrollo Super Destacado con información relevante:  652
contratado Desarrollo Destacado con información relevante:  3886
contratado Desarrollo Simple con información relevante:  54
contratado freemium con información relevante:  0


/home/fvadell/anaconda3/envs/fast/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


#### Caso columnas SD, S, D

Las tres columnas son simplemente la suma de otras dos así que vamos a sacarlas.

In [32]:
variables.remove('SD')
variables.remove('S')
variables.remove('D')

#### Agrego Columna Status Cliente pero numérica -> Esto lo tengo que ver después

In [33]:
data = data.sort_values(by = ['idclientecrm', 'ano_mes'], ascending = [False, False])

In [34]:
data['Status Cliente'] = np.where(data['Status Cliente']=='Cancelado', 1, 0)

In [35]:
label_references.loc[len(label_references)] = ['Cancelado', 1]
label_references.loc[len(label_references)] = ['Vigente', 0]

### Data Wrangling

In [36]:
# Estos casos particulares hay que corregirlos a mano
# 10455282 año 2019: Vigente
# 10726969 año 2019: Cancelado
# 30016421 año 2019: Vigente 
data = data[(data['idclientecrm']!=10455282)|(data['Status Cliente']==0)]
data = data[(data['idclientecrm']!=10726969)|(data['Status Cliente']==1)]
data = data[(data['idclientecrm']!=30016421)|(data['Status Cliente']==0)]

In [37]:
# Chequeamos si hay ids que aparezcan más de una vez por 'ano_mes'
ids_raros = pd.DataFrame(data.groupby(['idclientecrm', 'ano_mes']).size()).reset_index(drop = False).sort_values(0, ascending = False)
ids_raros = ids_raros[ids_raros[0]>1].idclientecrm.unique()
if len(ids_raros)>0:
    print('**** Revisar estos {} ids'.format(len(ids_raros)))
    print(ids_raros)
else:
    print("-------- OK!")

-------- OK!


In [38]:
variables.remove('mes_alta')
variables.remove('ano_mes')

In [39]:
label_references = generate_and_append_labels(data['provincia'].unique(), label_references)
data = apply_label(data, column_name = 'provincia', label_df = label_references)
variables.remove('provincia')
variables.append('provincia_label')

In [40]:
variables_temporales = variables.copy()

variables_temporales.remove('CLIENTE GRUPO_label')
variables_temporales.remove('Cliente Ilimitado sin Destaque')
variables_temporales.remove('Integrador_label')
variables_temporales.remove('Equipo')
variables_temporales.remove('provincia_label')

variables_fijas = list(set(variables).difference(set(variables_temporales)))

### Detectar cese de actividad

In [41]:
variables_actividad = ['monto_total', 'Lead Venta', 'Lead Alquiler', 'Lead Alquiler temporal']

In [42]:
data_original = data.copy()

In [43]:
data = data.reset_index(drop = True)

In [44]:
# Detectar cese de actividad a lo bruto
# TODO: MEJORAR ESTO
_ = data[(data['monto_total']<=0)&(data['Lead Venta']<=0)&(data['Lead Alquiler']<=0)&(data['Lead Alquiler temporal']<=0)].index
data = data.drop(_)

In [45]:
data = data.sort_values(['ano_alta', 'idclientecrm', 'mes_alta']).reset_index(drop = True)

In [46]:
clientes = data.sort_values(['idclientecrm', 'ano_mes'], ascending = [False, False]).filter(['idclientecrm', 'Status Cliente']).drop_duplicates('idclientecrm').sort_values('idclientecrm', ascending = True)

In [47]:
user_count = pd.DataFrame(data.idclientecrm.value_counts())
user_count = user_count.reset_index(drop = False)
user_count.columns = ['idclientecrm', 'months']

In [48]:
user_count = pd.merge(clientes, user_count)

In [49]:
def get_last_activity(data, user, n_months, drop_last_months = 0):
    res = data[data['idclientecrm']==user].sort_values(['ano_alta', 'mes_alta'], ascending = [False, False]).head(n_months+drop_last_months)
    res = res.drop(['pais', 'mes_alta', 'ano_mes'], axis = 1)
    res['month'] = [x for x in range(len(res))]
    return res.iloc[drop_last_months:,:]

### Aclaración
#### Detectar cese de actividad
Lo que se hace es sacar las columnas que tienen monto total 0 o negativo y sin Leads de ningún tipo. Esto nos ayuda a detectar el mes de cese de actividad y poder también calcular la cantidad de meses de actividad.
#### Problema
Como 'Status Cliente' se calcula a año cerrado tenemos un sesgo importante con mes_alta y ano_mes una vez que descartamos las filas sin actividad. Si calculamos la cantidad de meses de actividad, esta columna tendrá implícitamente información sobre el Status del Cliente. Por ejemplo, Si tenemos un usuario con un sólo mes de actividad y es de cualquier mes que no sea Diciembre entonces sabremos que el Status Cliente es Cancelado, es decir que esas dos columnas tienen información de Status Cliente. Esto es un ejemplo de __target leak__.
#### Solución
Si el modelo final va a tratar de predecir el Status del Cliente dados los últimos n meses de actividad entonces tendremos que entrenarlo con los últimos n meses de actividad de cada usuario (y descargar los usuarios que tengan menos de n meses de actividad)

## Primer DataSet transformado

In [50]:
valid_users = list(user_count[user_count['months']>=MIN_NUMBER_OF_ACTIVE_MONTHS].drop_duplicates('idclientecrm').idclientecrm)

In [51]:
n_users = len(valid_users)
df_v2 = pd.DataFrame()
for k, user in enumerate(valid_users):
    if (k % 100) == 0:
        print("{}/{}".format(k, n_users), end = "\r")
    _ = get_last_activity(data, user, n_months = MIN_NUMBER_OF_ACTIVE_MONTHS-ALERT_TIME, drop_last_months = ALERT_TIME)
    df_v2 = pd.concat([df_v2, _])
print("-------- OK!")

-------- OK!


In [52]:
df_v2 = df_v2.pivot(index='idclientecrm',columns='month',values=variables_temporales)

In [53]:
if len(clientes.idclientecrm.unique())== len(clientes):
    print("-------- OK!")
else:
    print("Hay algún cliente con más de un Status. Revisar !!")

-------- OK!


In [54]:
df_v2.columns =list(df_v2.columns.get_level_values(0)+df_v2.columns.get_level_values(1).astype(str))

## User info

In [55]:
client_data = data[['idclientecrm']+variables_fijas].drop_duplicates('idclientecrm')

In [56]:
client_data = client_data.sort_values('idclientecrm')

In [57]:
client_data.to_csv('./data/client_data.csv')

In [58]:
client_data = client_data.set_index('idclientecrm')

In [59]:
client_data = client_data.drop('Equipo', axis = 1)

In [60]:
df_v2 = pd.merge(df_v2, client_data, left_index = True, right_index = True)

In [61]:
clientes = clientes.set_index('idclientecrm')

In [62]:
clientes = clientes[clientes.index.isin(df_v2.index)]

In [63]:
df_v2.to_csv('./data/features.csv')
clientes.to_csv('./data/labels.csv')
label_references.to_csv('./data/label_references.csv')